In [6]:
import pandas as pd
import requests
import openai
import os
from dotenv import load_dotenv

In [12]:
# Load the environment variables from .env file
load_dotenv()

# Now you can access the environment variable just like before
openai_api_key = os.environ.get('OPENAI_API_KEY')